# Appendix 10.1: Chaining Prompts

- [Lesson](#lesson)
- [Example Playground](#example-playground)

## Setup

Run the following setup cell to load your API key and establish the `get_completion` helper function.

In [1]:
# Import python's built-in regular expression library
import re
import boto3
from botocore.exceptions import ClientError
import json

# Import the hints module from the utils package
from utils import hints

# Retrieve the MODEL_NAME variable from the IPython store
#modelId = 'anthropic.claude-3-5-sonnet-20241022-v2:0'
modelId = 'anthropic.claude-3-5-haiku-20241022-v1:0'

session = boto3.Session() # create a boto3 session to dynamically get and set the region name
region = session.region_name

bedrock_client = boto3.client(service_name='bedrock-runtime', region_name=region)

In [2]:
def get_completion(messages, system_prompt=None):
    inference_config = {
        "temperature": 0.5,
        "maxTokens": 200
    }
    converse_api_params = {
        "modelId": modelId,
        "messages": messages,
        "inferenceConfig": inference_config
    }
    if system_prompt:
        converse_api_params["system"] = [{"text": system_prompt}]
    try:
        response = bedrock_client.converse(**converse_api_params)
        text_content = response['output']['message']['content'][0]['text']
        return text_content

    except ClientError as err:
        message = err.response['Error']['Message']
        print(f"A client error occured: {message}")

---

## Lesson

The saying goes, "Writing is rewriting." It turns out, **Claude can often improve the accuracy of its response when asked to do so**!

There are many ways to prompt Claude to "think again". The ways that feel natural to ask a human to double check their work will also generally work for Claude. (Check out our [prompt chaining documentation](https://docs.anthropic.com/claude/docs/chain-prompts) for further examples of when and how to use prompt chaining.)

### Examples

In this example, we ask Claude to come up with ten words... but one or more of them isn't a real word.

In [3]:
# Initial prompt
first_user = "Name ten words that all end with the exact letters 'ab'."

# API messages array
messages = [
    {"role": "user",
     "content": [{"text": first_user}]}
]

# Store and print Claude's response
first_response = get_completion(messages)
print(first_response)

Here are ten words that end with 'ab':

1. cab
2. dab
3. fab
4. gab
5. lab
6. nab
7. sab
8. tab
9. jab
10. grab


**Asking Claude to make its answer more accurate** fixes the error! 

Below, we've pulled down Claude's incorrect response from above and added another turn to the conversation asking Claude to fix its previous answer.

In [4]:
second_user = "Please find replacements for all 'words' that are not real words."

# API messages array
messages = [
    {
        "role": "user",
        "content": [{"text": first_user}]
    },
    {
        "role": "assistant",
         "content": [{"text": first_response}]
    },
    {
        "role": "user",
         "content": [{"text": second_user}]
    },

]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

------------------------ Full messsages array with variable substutions ------------------------
[{'role': 'user', 'content': [{'text': "Name ten words that all end with the exact letters 'ab'."}]}, {'role': 'assistant', 'content': [{'text': "Here are ten words that end with 'ab':\n\n1. cab\n2. dab\n3. fab\n4. gab\n5. lab\n6. nab\n7. sab\n8. tab\n9. jab\n10. grab"}]}, {'role': 'user', 'content': [{'text': "Please find replacements for all 'words' that are not real words."}]}]

------------------------------------- Claude's response -------------------------------------
Here's a revised list with only real words:

1. cab
2. dab
3. fab (informal, short for "fabulous")
4. lab
5. nab
6. tab
7. jab
8. grab
9. scab
10. stab


But is Claude revising its answer just because we told it to? What if we start off with a correct answer already? Will Claude lose its confidence? Here, we've placed a correct response in the place of `first_response` and asked it to double check again.

In [7]:
first_user = "Name ten words that all end with the exact letters 'ab'."

first_response = """Here are 10 words that end with the letters 'ab':

1. Cab
2. Dab
3. Grab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Blab"""

second_user = "Please find replacements for all 'words' that are not real words."

# API messages array
messages = [
    {
        "role": "user",
        "content": [{"text": first_user}]
    },
    {
        "role": "assistant",
         "content": [{"text": first_response}]
    },
    {
        "role": "user",
         "content": [{"text": second_user}]
    },

]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

------------------------ Full messsages array with variable substutions ------------------------
[{'role': 'user', 'content': [{'text': "Name ten words that all end with the exact letters 'ab'."}]}, {'role': 'assistant', 'content': [{'text': "Here are 10 words that end with the letters 'ab':\n\n1. Cab\n2. Dab\n3. Grab\n4. Gab\n5. Jab\n6. Lab\n7. Nab\n8. Slab\n9. Tab\n10. Blab"}]}, {'role': 'user', 'content': [{'text': "Please find replacements for all 'words' that are not real words."}]}]

------------------------------------- Claude's response -------------------------------------
Here's a revised list of 10 words that end with 'ab', all of which are real words:

1. Cab
2. Dab
3. Grab
4. Gab (a real word meaning to talk idly)
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Stab

All of these are legitimate words in standard English dictionaries.


You may notice that if you generate a respnse from the above block a few times, Claude leaves the words as is most of the time, but still occasionally changes the words even though they're all already correct. What can we do to mitigate this? Per Chapter 8, we can give Claude an out! Let's try this one more time.

In [9]:
first_user = "Name ten words that all end with the exact letters 'ab'."

first_response = """Here are 10 words that end with the letters 'ab':

1. Cab
2. Dab
3. Grab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Blab"""

second_user = "Please find replacements for all 'words' that are not real words. If all the words are real words, return the original list."

# API messages array
messages = [
    {
        "role": "user",
        "content": [{"text": first_user}]
    },
    {
        "role": "assistant",
         "content": [{"text": first_response}]
    },
    {
        "role": "user",
         "content": [{"text": second_user}]
    },

]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

------------------------ Full messsages array with variable substutions ------------------------
[{'role': 'user', 'content': [{'text': "Name ten words that all end with the exact letters 'ab'."}]}, {'role': 'assistant', 'content': [{'text': "Here are 10 words that end with the letters 'ab':\n\n1. Cab\n2. Dab\n3. Grab\n4. Gab\n5. Jab\n6. Lab\n7. Nab\n8. Slab\n9. Tab\n10. Blab"}]}, {'role': 'user', 'content': [{'text': "Please find replacements for all 'words' that are not real words. If all the words are real words, return the original list."}]}]

------------------------------------- Claude's response -------------------------------------
After reviewing the list, all of these are real words:

1. Cab
2. Dab
3. Grab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Blab

These are all legitimate words in standard English dictionaries.


Try generating responses from the above code a few times to see that Claude is much better at sticking to its guns now.

You can also use prompt chaining to **ask Claude to make its responses better**. Below, we asked Claude to first write a story, and then improve the story it wrote. Your personal tastes may vary, but many might agree that Claude's second version is better.

First, let's generate Claude's first version of the story.

In [10]:
# Initial prompt
first_user = "Write a three-sentence short story about a girl who likes to run."

# API messages array
messages = [
    {
        "role": "user",
         "content": [{"text": first_user}]
    }
]

# Store and print Claude's response
first_response = get_completion(messages)
print(first_response)

Emma loved the feeling of wind rushing past her face as she sprinted down the country road, her feet pounding rhythmically against the dusty ground. Every morning before sunrise, she would lace up her worn running shoes and set out to explore the quiet landscape, finding freedom in the solitude of her early morning runs. As the sun began to peek over the distant hills, Emma smiled, knowing that these moments of pure motion and peace were what made her truly happy.


Now let's have Claude improve on its first draft.

In [11]:
second_user = "Make the story better."

# API messages array
messages = [
    {
        "role": "user",
        "content": [{"text": first_user}]
    },
    {
        "role": "assistant",
         "content": [{"text": first_response}]
    },
    {
        "role": "user",
         "content": [{"text": second_user}]
    },

]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

------------------------ Full messsages array with variable substutions ------------------------
[{'role': 'user', 'content': [{'text': 'Write a three-sentence short story about a girl who likes to run.'}]}, {'role': 'assistant', 'content': [{'text': 'Emma loved the feeling of wind rushing past her face as she sprinted down the country road, her feet pounding rhythmically against the dusty ground. Every morning before sunrise, she would lace up her worn running shoes and set out to explore the quiet landscape, finding freedom in the solitude of her early morning runs. As the sun began to peek over the distant hills, Emma smiled, knowing that these moments of pure motion and peace were what made her truly happy.'}]}, {'role': 'user', 'content': [{'text': 'Make the story better.'}]}]

------------------------------------- Claude's response -------------------------------------
Emma's legs burned with a familiar, welcome ache as she raced along the gravel road, her breath forming small cl

This form of substitution is very powerful. We've been using substitution placeholders to pass in lists, words, Claude's former responses, and so on. You can also **use substitution to do what we call "function calling," which is asking Claude to perform some function, and then taking the results of that function and asking Claude to do even more afterward with the results**. It works like any other substitution. More on this in the next appendix.

Below is one more example of taking the results of one call to Claude and plugging it into another, longer call. Let's start with the first prompt (which includes prefilling Claude's response this time).

In [12]:
first_user = """Find all names from the below text:

"Hey, Jesse. It's me, Erin. I'm calling about the party that Joey is throwing tomorrow. Keisha said she would come and I think Mel will be there too."""

prefill = "<names>"

# API messages array
messages = [
    {
        "role": "user",
        "content": [{"text": first_user}]
    },
    {
        "role": "assistant",
         "content": [{"text": prefill}]
    }
]

# Store and print Claude's response
first_response = get_completion(messages)
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(first_response)

------------------------ Full messsages array with variable substutions ------------------------
[{'role': 'user', 'content': [{'text': 'Find all names from the below text:\n\n"Hey, Jesse. It\'s me, Erin. I\'m calling about the party that Joey is throwing tomorrow. Keisha said she would come and I think Mel will be there too.'}]}, {'role': 'assistant', 'content': [{'text': '<names>'}]}]

------------------------------------- Claude's response -------------------------------------

- Jesse
- Erin
- Joey
- Keisha
- Mel
</names>


Let's pass this list of names into another prompt.

In [13]:
second_user = "Alphabetize the list."

# API messages array
messages = [
    {
        "role": "user",
        "content": [{"text": first_user}]
    },
    {
        "role": "assistant",
         "content": [{"text": prefill + "\n" + first_response}]
    },
    {
        "role": "user",
         "content": [{"text": second_user}]
    },

]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

------------------------ Full messsages array with variable substutions ------------------------
[{'role': 'user', 'content': [{'text': 'Find all names from the below text:\n\n"Hey, Jesse. It\'s me, Erin. I\'m calling about the party that Joey is throwing tomorrow. Keisha said she would come and I think Mel will be there too.'}]}, {'role': 'assistant', 'content': [{'text': '<names>\n\n- Jesse\n- Erin\n- Joey\n- Keisha\n- Mel\n</names>'}]}, {'role': 'user', 'content': [{'text': 'Alphabetize the list.'}]}]

------------------------------------- Claude's response -------------------------------------
<names>
- Erin
- Jesse
- Joey
- Keisha
- Mel
</names>


Now that you've learned about prompt chaining, head over to Appendix 10.2 to learn how to implement function calling using prompt chaining.

---

## Example Playground

This is an area for you to experiment freely with the prompt examples shown in this lesson and tweak prompts to see how it may affect Claude's responses.

In [ ]:
# Initial prompt
first_user = "Name ten words that all end with the exact letters 'ab'."

# API messages array
messages = [
    {"role": "user",
     "content": [{"text": first_user}]}
]

# Store and print Claude's response
first_response = get_completion(messages)
print(first_response)

In [ ]:
second_user = "Please find replacements for all 'words' that are not real words."

# API messages array
messages = [
    {
        "role": "user",
        "content": [{"text": first_user}]
    },
    {
        "role": "assistant",
         "content": [{"text": first_response}]
    },
    {
        "role": "user",
         "content": [{"text": second_user}]
    },

]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

In [ ]:
first_user = "Name ten words that all end with the exact letters 'ab'."

first_response = """Here are 10 words that end with the letters 'ab':

1. Cab
2. Dab
3. Grab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Blab"""

second_user = "Please find replacements for all 'words' that are not real words."

# API messages array
messages = [
    {
        "role": "user",
        "content": [{"text": first_user}]
    },
    {
        "role": "assistant",
         "content": [{"text": first_response}]
    },
    {
        "role": "user",
         "content": [{"text": second_user}]
    },

]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

In [ ]:
first_user = "Name ten words that all end with the exact letters 'ab'."

first_response = """Here are 10 words that end with the letters 'ab':

1. Cab
2. Dab
3. Grab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Blab"""

second_user = "Please find replacements for all 'words' that are not real words. If all the words are real words, return the original list."

# API messages array
messages = [
    {
        "role": "user",
        "content": [{"text": first_user}]
    },
    {
        "role": "assistant",
         "content": [{"text": first_response}]
    },
    {
        "role": "user",
         "content": [{"text": second_user}]
    },

]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

In [ ]:
# Initial prompt
first_user = "Write a three-sentence short story about a girl who likes to run."

# API messages array
messages = [
    {
        "role": "user",
         "content": [{"text": first_user}]
    }
]

# Store and print Claude's response
first_response = get_completion(messages)
print(first_response)

In [ ]:
second_user = "Make the story better."

# API messages array
messages = [
    {
        "role": "user",
        "content": [{"text": first_user}]
    },
    {
        "role": "assistant",
         "content": [{"text": first_response}]
    },
    {
        "role": "user",
         "content": [{"text": second_user}]
    },

]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

In [ ]:
first_user = """Find all names from the below text:

"Hey, Jesse. It's me, Erin. I'm calling about the party that Joey is throwing tomorrow. Keisha said she would come and I think Mel will be there too."""

prefill = "<names>"

# API messages array
messages = [
    {
        "role": "user",
        "content": [{"text": first_user}]
    },
    {
        "role": "assistant",
         "content": [{"text": prefill}]
    }
]

# Store and print Claude's response
first_response = get_completion(messages)
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(first_response)

In [ ]:
second_user = "Alphabetize the list."

# API messages array
messages = [
    {
        "role": "user",
        "content": [{"text": first_user}]
    },
    {
        "role": "assistant",
         "content": [{"text": prefill + "\n" + first_response}]
    },
    {
        "role": "user",
         "content": [{"text": second_user}]
    },

]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))